# Detección de anomalias
**Proyecto:** 3  
**Equipo:**  
    -Ibsan Otniel Morales Yepiz  
    -Hugo de Jesús Valenzuela Chaparro  
    -Héctor Rodríguez Domínguez  
    
### Descripción de Datos
   -El dataset empleado para este proyecto se obutvo de la plataforma de Kaggle en el siguiente enlace: https://www.kaggle.com/tangodelta/api-access-behaviour-anomaly-dataset
    
   -El dataset contine las sisguientes columnas:  
    
   <ul>
    <li>**inter_api_access_duration(sec):** Intervalo de tiempo entre dos accesos consecutivos en la sesión de un usuario  </li>
    <li>**api_access_uniqueness:** La proporción de diferentes APIs utilzadas con relación a las APIs utilizadas en la sesión de un usuario  </li>
    <li>**sequence_length(count):** Promedio de llamadas que un usuario hace en una sesión  </li>
    <li>**vsession_duration(min):** Duración de una sesión en minutos   </li>
    <li>**ip_type:** Tipo de ip de donde proviene el usuario  </li>
    <li>**behavior:** subtipo de comportamiento  </li>
    <li>**behavior_type:** Tipo de comportamiento  </li>
    <li>**num_sessions:** Numeor de sesiónes con diferente id de sesión  </li>
    <li>**num_users:** NUmero de usuarios realizando el mismo tipo de llamadas a la API  </li>
    <li>**num_unique_apis:** Número de APIs distintas en el mismo grupo de ocmportamiento  </li>
    <li>**source:** Origen de la información  </li>
    </ul>
    

### Importando librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import seaborn as sns
from pandas_profiling import ProfileReport 
from sklearn.neighbors import LocalOutlierFactor
import sweetviz

### Lectura y preparación de datos

In [ ]:
#Read file
df = pd.read_csv("data/remaining_behavior_ext.csv")

#Rename columns according to our preference
df.rename(
    columns={"inter_api_access_duration(sec)": "access_duration", "api_access_uniqueness": "access_uniqueness",
             "sequence_length(count)": "session_calls", "vsession_duration(min)": "session_duration" }, 
    inplace=True
)

#Pop useless columns for our intention
df.pop("ip_type")
df.pop("behavior")
df.pop("num_sessions")
df.pop("num_unique_apis")
df.pop("source")

#we remove unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

#We assign a nuemrical value to behavior_type column
#df["behavior_type"].replace({"attack": 1, "bot": 2, "normal": 3, "outlier": 4}, inplace=True)

#We transform minutes to seconds in order to normalize the DF
df["session_duration"] = 60 * df["session_duration"]


df.head()

### Análisis exploratorio
En esta sección se revisa la información de la estrucutra general de nuestro dataframe, asi como datos generales de este.  

**La forma de nuestro dataframe:** 

In [5]:
df.shape

NameError: name 'df' is not defined

**Tipos de datos:**  

In [ ]:
df.dtypes

**Valores perdidos, repetidos y NaN:**  

In [ ]:
#Valores vacios
df.isnull().sum()

In [ ]:
#We remove rows with null values
df = df.dropna()
print("Dataset despues de quitar valores nulos")
df.isnull().sum()

In [ ]:
#Eliminar valores NaN
df.dropna(inplace=True)

#Valores duplicados
print(df.duplicated().sum())

In [ ]:
#Eliminamos filas con valores duplicados
df.drop_duplicates(keep='last',inplace=True)
print(df.duplicated().sum())

**Mapa de correlación**  
En esta sección se muestra un mapa de calor, donde podemos observar que no hay una correlación importante entre las diferentes columnas del dataset



In [ ]:
plt.figure(figsize=(20,6))
hm = sns.heatmap(df.corr(),vmin=-1, vmax=1, annot=True,cmap='PiYG')
hm.set_title('Correlation Heatmap',  fontdict={'fontsize':18});
plt.show()

In [ ]:
sns.pairplot(df) 
plt.show()

#### Descripción de datos

In [ ]:
df.groupby('behavior_type').agg(['mean','min', 'max'])

#### Creación de perfil con "sweetviz"

In [ ]:
df_profile = ProfileReport(
    df, 
    explorative=True,
    title='Comportamiento de dataframe', 
    html={'style':{'full_width':False}}
) 

df_profile.to_file("profile.html")

B_profile = sweetviz.analyze(df)
B_profile.show_html("profile.html")

#### Basados en la información previamente obtenida en el análisis del dataset, queremos detectar las anomalias en el tiempo que transcurre entre cada llamada a una API, para determinar si es un bot.

#### Local Outlier factor

El factor atípico local es un algoritmo propuesto para encontrar puntos de datos anómalos midiendo la desviación local de un punto de datos dado con respecto a sus vecinos.

In [ ]:
df["behavior_type"].replace({"attack": 1, "bot": 2, "normal": 3, "outlier": 4}, inplace=True)

In [ ]:
clf = LocalOutlierFactor(n_neighbors=100)
estimado_LOF = clf.fit_predict(df)

plt.scatter(x=df.iloc[:,0], y=df.iloc[:,1], c=np.where(estimado_LOF > 0.0,1,0), cmap='jet')
#plt.scatter(x=X[:,0], y=X[:,1], c=estimado_LOF, cmap='jet')
plt.xlabel("num_users")
plt.ylabel("session_duration")
plt.colorbar()
plt.show()

In [ ]:
np.where(estimado_LOF > 0.0,1,0).mean()

#### Isolation Forest
Es una método no supervisado para identificar anomalías (outliers) cuando los datos no están etiquetados, es decir, no se conoce la clasificación real (anomalía - no anomalía) de las observaciones. 

Su funcionamiento está inspirado en el algoritmo de clasificación y regresión Random Forest

In [ ]:
random_state = np.random.RandomState(43)
model=IsolationForest(n_estimators=50, contamination=float(0.9), warm_start=True, random_state=random_state)

#model = IsolationForest(n_estimators=50, warm_start=True)
model.fit(df[['access_duration']])

df['scores'] = model.decision_function(df[['access_duration']])

df['anomaly_score'] = model.predict(df[['access_duration']])

#print(df[df['anomaly_score']==-1].head(20))
#df[df['anomaly_score']==-1].tail(20) 

anomaly_count = df.shape[0]
    
accuracy = 100*list(df['anomaly_score']).count(-1)/anomaly_count
#accuracy = df[df['anomaly_score']==-1].head()
print("Accuracy of the model:", accuracy)

plt.scatter(x=df.iloc[:,0], y=df.iloc[:,1], c=df['scores'], cmap='jet')
plt.colorbar()
plt.show()

#### Conclusiones

Como se puede observar, este dataset presenta casos muy especiales donde los valores cercanos al 0 se presentan como valores anomales fdebido a la rapides de acceso entre las aplicaciones

Asimismo no existe mucha correlacion entre las diferente variables, por lo cual se decidio trabajar con la columna de "access_duration", la cual tiene una mayor correlación con "session_duration" como se muestra en los pair plots.

Contemplando los resultados entre ambos metodos, se puede observar que el mejor resultado se obtiene a traves del uso del Local Outlier factor. debido a que se ajusta mejor por los vecinos y cercania de los valores en de

***

# Conjunto de Datos con etiquetado
En esta siguiente sección se usa el otro dataset, llamado ```supervised_dataset```, el cual contiene ya etiquetado si es outlier o no. Sin embargo en este proyecto se ignora el etiquetado y se aplican los métodos de detección de anomalías como si no se tuviesen.

### Diccionario de los datos
- **inter_api_access_duration(sec):** Intervalo de tiempo entre dos accesos consecutivos a la API en una sesión de usuario.
- **api_access_uniqueness:** La proporción del número de APIs distintas vistas en una sesión de un usuario con respecto al total de llamadas hechas a la API en esa sesión.
- **sequence_length(count):** El número total de llamadas a API hechas en una sesión por una usuario en promedio.
- **vsession_duration(min):** La duración de una sesión de usuario dentro de una ventana de observación en minutos.
- **ip_type:** El tipo de IP de donde proviene el usuario.
- **num_sessions:** Número de sesiones de usuario cada una con diferente session_id.
- **num_users:** Número de usuarios generando el mismo tipo de secuencias de llamadas de API.
- **num_unique_apis:** Número de APIs distintas en ese grupo de comportamiento (behavior group).
- **source:** Origen de los datos. F = Financial services, E = Ecommerce.
- **classification:** Clasificación, outlier o normal.

# Local Outlier Factor

## CARGA DE DATOS

In [ ]:
dfsup = pd.read_csv('/data/supervised_dataset.csv')

In [ ]:
dfsup

## ANALISIS EXPLORATORIO

### ESTRUCTURA DE DATOS

In [ ]:
dfsup.shape

In [ ]:
#Estructura de datos
dfsup.dtypes

In [ ]:
dfsup.describe()

### Valores Perdidos

In [ ]:
dfsup.isnull().sum()

In [ ]:
dfsup.isna().sum()

In [ ]:
#Drop Nans
dftidy = dfsup.dropna()
#Drop Unnamed classification column
dftidy =dftidy.drop(['classification'], axis=1)
dftidy =dftidy.drop(dftidy.columns[[0]], axis=1)
#Make Columns type Factor
dftidy['source'] = dftidy['source'].astype('category')
dftidy['ip_type'] = dftidy['ip_type'].astype('category')
#dftidy.isna().sum()
dftidy.dtypes

In [ ]:
dftidy.shape

In [ ]:
plt.figure(figsize=(16,6))
sns.heatmap(dftidy.corr(),vmin=-1, vmax=1, annot=True,cmap='RdBu')
heatmap.set_title('Correlacion Heatmap', fontdict={'fontsize':18}, pad=12);
plt.show()

In [ ]:
sns.pairplot(dftidy) 
plt.show()

## LOCAL OUTLIER FACTOR

In [ ]:
X = dftidy[["num_users", "inter_api_access_duration(sec)"]].to_numpy()
X.shape

In [ ]:
clf = LocalOutlierFactor(n_neighbors=100)
estimado_LOF = clf.fit_predict(X)

plt.scatter(x=X[:,0], y=X[:,1], c=np.where(estimado_LOF > 0.0,1,0), cmap='jet')
#plt.scatter(x=X[:,0], y=X[:,1], c=estimado_LOF, cmap='jet')
plt.xlabel("num_users")
plt.ylabel("inter_api_access_duration(sec)")
plt.colorbar()
plt.show()

In [ ]:
np.where(estimado_LOF > 0.0,1,0).mean() 

***

# One-Class Support Vector Machine

## Leyendo datos

In [ ]:
# leyendo datos
df_sup = pd.read_csv("./data/supervised_dataset.csv", index_col=0)

### Descripción de las variables


In [ ]:
df_sup.dtypes

In [ ]:
# cambiando la variable vsession_duration a segundos para que sea comparable
df_sup.rename(
    columns={"vsession_duration(min)": "vsession_duration(sec)" }, 
    inplace=True
)
df_sup['vsession_duration(sec)'] = df_sup['vsession_duration(sec)'] * 60

In [ ]:
# categorizando las variables de categoria
for col in ['ip_type', 'source', 'classification']:
    df_sup[col] = df_sup[col].astype('category')
df_sup.dtypes

In [ ]:
# descripción de las características numéricas
df_sup.describe()

In [ ]:
# No hay valores perdidos
df_sup.isna().sum()

In [ ]:
# se retiran los valores duplicados
df_sup.duplicated().sum()

In [ ]:
df_sup = df_sup.drop_duplicates()
df_sup.duplicated().sum()

### Visualizaciones

In [ ]:
# dataframe con solo columnas numericas
df_sup_num = df_sup.drop(['ip_type','source', 'classification'], axis=1)

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df_sup, title="Pandas Profiling Report")

In [ ]:
profile.to_notebook_iframe()

No se observa normalidad en las columnas numéricas, por lo que no sirviría el método de curva elíptica.

Veamos primero un par de interacciones de variables.

In [ ]:
# numero de usuarios vs numero de sesion
sns.scatterplot(data=df_sup_num, x = 'num_users', y = 'num_sessions')

In [ ]:
df_sup_num.head()

In [ ]:
df_sup_num.dtypes

In [ ]:
sns.pairplot(data=df_sup_num)

## Detección de Anomalías

### One-Class Support Vector Machine

In [ ]:
# importar el modulo desde scikit learn
from sklearn.svm import OneClassSVM

In [ ]:
# ajustar el hiperparametro
clf = OneClassSVM(nu=0.15, kernel='rbf')

In [ ]:
# predicciones de los outliers
# el método regresa 1 para no outliers y -1 para outliers
y_outlier = clf.fit_predict(df_sup_num)

In [ ]:
y_outlier.shape

In [ ]:
np.unique(y_outlier, return_counts=True)

In [ ]:
# agregar la columna con las predicciones a nuevo dataframe, codificadas como outlier, normal
df_pred = df_sup_num.copy()
df_pred['y_outlier'] = np.where(y_outlier > 0, 'normal', 'outlier')
df_pred['y_outlier'] = df_pred['y_outlier'].astype('category')

In [ ]:
df_pred.head()

### Graficación de los outliers

In [ ]:
sns.pairplot(data=df_pred, hue = 'y_outlier')

In [ ]:
df_pred.dtypes

### Considerando solamente 4 características en el ajuste del One-Class SVM

- **inter_api_access_duration(sec):** Intervalo de tiempo entre dos accesos consecutivos a la API en una sesión de usuario.
- **sequence_length(count):** El número total de llamadas a API hechas en una sesión por una usuario en promedio.

- **num_sessions:** Número de sesiones de usuario cada una con diferente session_id.
- **num_users:** Número de usuarios generando el mismo tipo de secuencias de llamadas de API.

#### El dataframe

In [ ]:
# dataframe con las 4 características
df_subset = df_sup_num.copy()
df_subset = df_subset[['num_users','num_sessions', 'sequence_length(count)', 'inter_api_access_duration(sec)']]
#df_subset = df_subset[['num_users','num_sessions', 'inter_api_access_duration(sec)']]
#df_subset = df_subset[['num_users','num_sessions']]
df_subset.head()

In [ ]:
# ajustar el hiperparametro
clf = OneClassSVM(nu=0.15, kernel='rbf')
# predicciones de los outliers
# el método regresa 1 para no outliers y -1 para outliers
y_outlier = clf.fit_predict(df_subset)

In [ ]:
y_outlier.shape

In [ ]:
np.unique(y_outlier, return_counts=True)

In [ ]:
# agregar la columna con las predicciones a nuevo dataframe, codificadas como outlier, normal
df_pred = df_sup_num.copy()
df_pred['y_outlier'] = np.where(y_outlier > 0, 'normal', 'outlier')
df_pred['y_outlier'] = df_pred['y_outlier'].astype('category')

In [ ]:
df_pred.head()

In [ ]:
sns.pairplot(data=df_pred, hue = 'y_outlier')

### Considerando solamente 2 características en el ajuste del One-Class SVM

- **num_sessions:** Número de sesiones de usuario cada una con diferente session_id.
- **num_users:** Número de usuarios generando el mismo tipo de secuencias de llamadas de API.

In [ ]:
# dataframe con las 4 características
df_subset = df_sup_num.copy()
df_subset = df_subset[['num_users','num_sessions']]
df_subset.head()

# ajustar el hiperparametro
clf = OneClassSVM(nu=0.15, kernel='rbf')
# predicciones de los outliers
# el método regresa 1 para no outliers y -1 para outliers
y_outlier = clf.fit_predict(df_subset)

In [ ]:
y_outlier.shape

In [ ]:
np.unique(y_outlier, return_counts=True)

In [ ]:
# agregar la columna con las predicciones a nuevo dataframe, codificadas como outlier, normal
df_pred = df_sup_num.copy()
df_pred['y_outlier'] = np.where(y_outlier > 0, 'normal', 'outlier')
df_pred['y_outlier'] = df_pred['y_outlier'].astype('category')

df_pred.head()

In [ ]:
sns.pairplot(data=df_pred, hue = 'y_outlier')

## Conclusiones
como es de esperarse, al aplicar el método OC-SVM a las 7 variables numéricas, podemos ver que en todas las gráficas de pares se encuentran outliers. Mientras que se van considerando menos variables para la filtración de los outliers, se encuentran menos.

En este caso se fijo el parámetro ```nu = 0.15```  para considerar la misma proporción de outliers en los 3 casos.  

Considerando solo las variables de número de sesiones y número de usuarios vemos que se quedan algunas gráficas de pares sin filrar datos outliers.

El caso equilibrado parece ser considerar las variables de tiempo junto con las de sesiones, que son las siguientes

- **inter_api_access_duration(sec)** 
- **sequence_length(count)**
- **num_sessions** 
- **num_users** 

Parece filtrar bien como se observa en las gráficas de número de sesion y número de usuario contra el número de llamads únicas a la API, dejando un comportamiento que se ajustaría bien con una regresión lineal. 

Por otro lado, para la gráfica de número de usuarios contra número de sesiones parece filtrar excesivamente y romper la tendencia de linealidad que se observa.